<img src='Titanic_Portside_Diagram.jpg' height="1000px" width="80%">

In [53]:
import pandas as pd 
import numpy as np

In [54]:
data = pd.read_csv("train.csv")

In [84]:
data.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


<div> 
survival - Survival (0 = No; 1 = Yes)<br>
class - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)<br>
sex - Sex<br>
age - Age<br>
sibsp - Number of Siblings/Spouses Aboard<br>
parch - Number of Parents/Children Aboard<br>
ticket - Ticket Number<br>
fare - Passenger Fare<br>
cabin - Cabin<br>
embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)<br>
boat - Lifeboat (if survived)<br>
body - Body number (if did not survive and body was recovered)<br>
</div>

# 

In [85]:
data.shape

(418, 11)

In [86]:
data.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [87]:
data.drop(['SibSp', 'Parch', 'Name', 'Ticket', 'Cabin'], axis=1, inplace= True)

In [88]:
data.Age.describe()

count    332.000000
mean      30.272590
std       14.181209
min        0.170000
25%       21.000000
50%       27.000000
75%       39.000000
max       76.000000
Name: Age, dtype: float64

In [89]:
c = 0 
li = []
for u in data.Age.isna():
    if u == True :
        li.append(c)
    c+=1

In [90]:
g = data.groupby(['Pclass', 'Sex']).agg({'Age':'mean'})
g

Age
Pclass Sex              
1      female  41.333333
       male    40.520000
2      female  24.376552
       male    30.940678
3      female  23.073400
       male    24.525104

In [91]:
ages = g.Age.tolist()

In [92]:
_class = [1, 2, 3]
sex = ['female', 'male']
for j in _class:
    for i in sex:
        data.Age[(data.Sex == i) & (data.Pclass == j)] = data.Age[(data.Sex == i) & (data.Pclass == j)].fillna(ages.pop(0))

C:\Users\10\AppData\Local\Temp/ipykernel_15940/2957573702.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Age[(data.Sex == i) & (data.Pclass == j)] = data.Age[(data.Sex == i) & (data.Pclass == j)].fillna(ages.pop(0))


In [93]:
data.Age.isna().sum()

0

In [94]:
data.Fare.fillna(data.Fare.mean(), inplace= True)

In [95]:
data.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked
0,892,3,male,34.5,7.8292,Q
1,893,3,female,47.0,7.0000,S
2,894,2,male,62.0,9.6875,Q
3,895,3,male,27.0,8.6625,S
4,896,3,female,22.0,12.2875,S


In [96]:
df = pd.get_dummies(data['Embarked'])
sexDf = pd.get_dummies(data['Sex'])

In [97]:
newData = pd.concat([data, sexDf, df], axis= 1)

In [98]:
newData.drop(['Sex', 'Embarked'], axis=1, inplace= True)

# 

# make a column to specify if he was alone or not

# model

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [100]:
norData = normalize(newData[['Fare', 'Age']])

In [101]:
norData = pd.DataFrame(norData, columns=['fare', 'age'])

In [102]:
finalData = pd.concat([newData, norData], axis= 1)

In [103]:
finalData.drop(['Age', 'Fare'], axis=1, inplace= True)

In [104]:
finalData

,PassengerId,Pclass,female,male,C,Q,S,fare,age
0,892,3,0,1,0,1,0,0.221306,0.975204
1,893,3,1,0,0,0,1,0.147311,0.989090
2,894,2,0,1,0,1,0,0.154377,0.988012
3,895,3,0,1,0,0,1,0.305495,0.952194
4,896,3,1,0,0,0,1,0.487621,0.873055
...,...,...,...,...,...,...,...,...,...
413,1305,3,0,1,0,0,1,0.311865,0.950126
414,1306,1,1,0,1,0,0,0.941448,0.337158
415,1307,3,0,1,0,0,1,0.185059,0.982727
416,1308,3,0,1,0,0,1,0.311865,0.950126


In [76]:
X = finalData.drop('Survived', axis= 1)
y = finalData['Survived']

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

# 

In [78]:
from sklearn.linear_model import LogisticRegression

In [79]:
re = LogisticRegression()

In [80]:
re.fit(X_train, y_train)

LogisticRegression()

In [81]:
re.score(X_train, y_train)

0.7752808988764045

In [82]:
re.score(X_test, y_test)

0.8268156424581006

In [83]:
data = pd.read_csv('test.csv')

In [105]:
y_pred = re.predict(finalData)

In [115]:
x = y_pred.tolist()

In [116]:
y = finalData.PassengerId.tolist()

In [128]:
df = pd.DataFrame({'PassengerId':y, 'Survived':x})

In [130]:
df.set_index('PassengerId', inplace= True)

In [131]:
df

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [132]:
df.to_csv('kaggle.csv')